In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install torch-lr-finder
import torch
from PIL import Image
import os
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader
import torchvision.models as models
import albumentations as A
import cv2
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_lr_finder import *
from torch_lr_finder import LRFinder
import matplotlib.pyplot as plt
import torchvision.models as models
from sklearn.model_selection import train_test_split
from numpy import asarray 

In [ ]:
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
        os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/resnet101/resnet101.pth' '/root/.cache/torch/hub/checkpoints/resnet101-333f7ec4.pth'

In [ ]:
    
def train_tfm(size):
    return A.Compose([
    #             A.RandomResizedCrop(128, 128),
                A.Resize(size + size//10, size + size//10),
                A.RandomCrop(size, size),
                A.Transpose(p=0.5),
                A.HorizontalFlip(p=0.5),
                A.VerticalFlip(p=0.5),
                A.ShiftScaleRotate(p=0.5),
                A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
                A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
                A.CoarseDropout(p=0.5),
                A.Cutout(p=0.5),
                #A.ToTensorV2(p=1.0),
            ], p=1.)

      

def valid_tfm(size):
    return A.Compose([ A.Resize(size, size),
                     A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0)
                     ])



In [ ]:
def train_(epocs, loader, model, valid_set):
    best_v_acc = 0.0
    for epoch in range(epocs):  # loop over the dataset multiple times
        print("Training...")
        correct = 0
        total = 0
        running_loss = 0.0
        for i, data in enumerate(loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to('cuda'), labels.to('cuda')

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss=criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            max_index = outputs.max(dim = 1)[1]
            correct += (max_index == labels).sum().item()
            total += labels.size(0)

            
            # print statistics
            running_loss += loss.item()
            if i % len(loader) == len(loader) - 1:    
                print('[epoc: %d, %5d] training loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / len(loader)))
                running_loss = 0.0
        print("Training Accuracy = {}".format(100 * correct / total))
        
        
        correct = 0
        total = 0
        running_loss = 0.0
        print("Validating...")
        for i, data in enumerate(valid_set, 0):
            
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to('cuda'), labels.to('cuda')
                outputs = model(inputs)
                loss=criterion(outputs, labels)
                max_index = outputs.max(dim = 1)[1]
                correct += (max_index == labels).sum().item()
                total += labels.size(0)
                
                # print statistics
                running_loss += loss.item()
                if i % len(valid_set) == len(valid_set) - 1:    
                    print('[epoc: %d, %5d] valid loss: %.3f' %
                          (epoch + 1, i + 1, running_loss / len(loader)))
                    running_loss = 0.0
                    
        if (100 * correct / total) > best_v_acc:
            best_v_acc = (100 * correct / total)
            print("best model saved")
            torch.save(model.state_dict(), 'best_model.pt')
            
                
        print("valid Accuracy = {}".format(100 * correct / total))
                
                
    print('Finished Training')


In [ ]:
df = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')

In [ ]:
X, y = df['image_id'].values, df['label'].values
X_train, X_valid, Y_train, Y_valid = train_test_split(X, y, test_size=0.2)


In [ ]:
class CassDataset(Dataset):
    def __init__(self, Dir, FNames, Labels, Transform):
        self.dir = Dir
        self.fnames = FNames
        self.transform = Transform
        self.lbs = Labels
        
    def __len__(self):
        return len(self.fnames)

    def __getitem__(self, index):
        x = Image.open(os.path.join(self.dir, self.fnames[index]))
#         if self.valid:
#             res  = self.transform(image=asarray(x))
#             image = res['image'].astype(np.float32)
#             return image, self.lbs[index] 
        if "train" in self.dir: 
            res  = self.transform(image=asarray(x))
            image = res['image'].astype(np.float32)
            return image, self.lbs[index]            
        elif "test" in self.dir:            
            return self.transform(image=asarray(x)), self.fnames[index]

In [ ]:
def custom_collate(batch):
    data = torch.stack([torch.tensor(item[0]) for item in batch]).permute(0, 3, 1, 2) # fixes axes before feeding no nn.
    target = torch.LongTensor([item[1] for item in batch])  

    return data, target

In [ ]:
validate_tfm = valid_tfm(256)
training_tfm = train_tfm(256)

cass_trainset = CassDataset('../input/cassava-leaf-disease-classification/train_images/', X_train, Y_train, training_tfm)
cass_trainloader = DataLoader(cass_trainset, batch_size=64, shuffle=True, num_workers=4, collate_fn=custom_collate)
cass_validset = CassDataset('../input/cassava-leaf-disease-classification/train_images/', X_valid, Y_valid, validate_tfm)
cass_validloader = DataLoader(cass_validset, batch_size=64, shuffle=True, num_workers=4, collate_fn=custom_collate)


Lets look at some of the samples

In [ ]:
def visualize(image):
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    #plt.cla()

for i in range(10):
    visualize(cass_trainset[i][0][:])

## k, lets train!

In [ ]:
model_ft = models.resnet34(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 5)
criterion = nn.CrossEntropyLoss()
model_ft = model_ft.to('cuda')

In [ ]:

#optimizer = optim.Adam(model_ft.parameters(), lr=1e-7, weight_decay=1e-2)
optimizer = optim.SGD(model_ft.parameters(), lr=1e-7, momentum=0.9)
lr_finder = LRFinder(model_ft, optimizer, criterion)
lr_finder.range_test(cass_trainloader, end_lr=100, num_iter=100)
lr_finder.plot()
lr_finder.reset()

In [ ]:
optimizer = optim.SGD(model_ft.parameters(), lr=1e-2, momentum=0.9)

In [ ]:
train_(13, cass_trainloader, model_ft, cass_validloader)